# Домашнее задание 2

In [79]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import optuna

import warnings
warnings.filterwarnings('ignore')

## Загрузка данных и генерация датасета

In [80]:
measurement_step = 100
series_len = 10
files = glob.glob('datasets/*.csv')


def get_dataset(measurement_step, series_len):
    files = glob.glob('dataset/*/*.csv')

    measurement_step = 10

    data = []
    for file in files:
        if(file != 'data_model.csv'):
            temp = pd.read_csv(file, usecols = ['u1', 'u2', 'y1', 'y2', 'y_nominal1', 'y_nominal2',
                                                'fault_u1', 'fault_u2', 'fault_y1', 'fault_y2', 'comp_fault'])
            indexes = range(0, temp.shape[0], measurement_step)
            temp = temp.iloc[indexes][:]
            #temp.reset_index(inplace=True)
            data.append(temp)      

    #Уберем строки, не относящиеся к отказу ИНС и заменим ненулевые значения отказов на 1.
    drop_cols = ['fault_u1', 'fault_u1', 'fault_u2', 'comp_fault']
    for experiment in data:
        experiment.drop(drop_cols, inplace=True, axis=1)
        experiment['fault_y1'].replace([1., 2., 3,], 1, inplace=True)
        experiment['fault_y2'].replace([1., 2., 3,], 1, inplace=True)
        
    #Заменим абсолютные значения выходов на отклонения
    for experiment in data:
        experiment['dev_y1'] = experiment['y_nominal1'] - experiment['y1']
        experiment['dev_y2'] = experiment['y_nominal2'] - experiment['y2']
        experiment.drop(['y1', 'y2', 'y_nominal1', 'y_nominal2'], inplace=True, axis=1)
        
    #Добавим первую и вторую производные
    for df1 in data:
        df1['V_dev_y1'] = df1['dev_y1'].shift(1) - df1['dev_y1']
        df1['V_dev_y2'] = df1['dev_y2'].shift(1) - df1['dev_y2']
        df1.dropna(axis=0, inplace=True)
        df1['A_dev_y1'] = df1['V_dev_y1'].shift(1) - df1['V_dev_y1']
        df1['A_dev_y2'] = df1['V_dev_y2'].shift(1) - df1['V_dev_y2']
        df1.dropna(axis=0, inplace=True)
        
    # Добавим скользящие средние на series_len и series_len/2
    cols = [col for col in data[0].columns if ('y1' in col or
                                        'y2' in col or
                                        'u1' in col) and 'fault' not in col]
    for df1 in data:
        for col in cols:
            df1['roll_5_' + col] = df1[col].rolling(series_len // 2).mean()
            df1['roll_10_' + col] = df1[col].rolling(series_len).mean()
        df1.dropna(axis=0, inplace=True)
        df1.reset_index(drop=True, inplace=True)
        
    #Сформируем датасет в виде временных последовательностей по series_len значений
    lag = series_len - 1
    new_cols = []
    for col in data[0].drop(['fault_y1', 'fault_y2'], axis=1).columns:
        new_cols += [col + '_' + str(i) for i in range(series_len)]
    new_cols += ['fault_y1', 'fault_y2']

    df = pd.DataFrame(data=None, columns=new_cols)
    cols = data[0].drop(['fault_y1', 'fault_y2'], axis=1).columns

    for k, experiment in enumerate(data):
        for i in range(lag, experiment.shape[0]):
            sample = []
            for col in cols:
                temp = list(experiment.loc[(i-lag):i][col])
                sample += temp
                
            sample_fault_y1 = list(experiment.loc[(i-lag):i]['fault_y1'])
            if sum(sample_fault_y1) == series_len:
                fault_y1_data = 1
            else:
                fault_y1_data = 0
                
            sample_fault_y2 = list(experiment.loc[(i-lag):i]['fault_y2'])
            if sum(sample_fault_y2) == series_len:
                fault_y2_data = 1
            else:
                fault_y2_data = 0
                
            sample += [fault_y1_data, fault_y2_data]
            df.loc[len(df)] = sample
            
    df['fault_LE'] = 0
    df['fault_LE'].loc[ (df['fault_y1'] == 0) & (df['fault_y1'] == 0)] = 0
    df['fault_LE'].loc[df['fault_y1'] == 1] = 1
    df['fault_LE'].loc[df['fault_y2'] == 1] = 2

    scaler = StandardScaler()
    X_array = scaler.fit_transform(df.drop(['fault_y1', 'fault_y2', 'fault_LE'], axis=1))
    y_le = np.array(df['fault_LE'])

    X_series = X_array.reshape((X_array.shape[0], series_len, int(X_array.shape[1] / series_len)))
    y_ohe = np.array(df[['fault_y1', 'fault_y2']])

    return df, X_array, y_le, X_series, y_ohe 

In [81]:
df, X_array, y_le, X_series, y_ohe = get_dataset(measurement_step, series_len)

X_array_train, X_array_test, y_le_train, y_le_test = train_test_split(X_array, y_le, stratify=y_le, test_size=0.25, random_state=0)

In [82]:
X_array_train.shape, y_le_train.shape

((25200, 220), (25200,))

In [83]:
col_names = df.drop(['fault_y1', 'fault_y2', 'fault_LE'], axis=1).columns

X_train, X_test = pd.DataFrame(X_array_train, columns=col_names), pd.DataFrame(X_array_test, columns=col_names)
y_train, y_test = pd.DataFrame(y_le_train, columns=['fault']), pd.DataFrame(y_le_test, columns=['fault'])
X_train.shape, y_train.shape

((25200, 220), (25200, 1))

## Построение модели
В ходе выоплнения первого задания достаточно высокую точность показал случайный лес при низких вычислительных затратах в сравнении с аналогами по выбранной метрике. Проведем подбор его гиперпараметров с применением optuna

In [84]:
def objective(trial, X, y, cv):

  params = params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        }
  rf_clf = RandomForestClassifier(**params, random_state=0)
  cv_result = cross_validate(rf_clf, X, y, cv = cv, scoring=['precision_macro', 'recall_macro'], n_jobs = -1)
  precision = cv_result ["test_precision_macro"].mean()
  recall = cv_result ["test_recall_macro"].mean()
  f1 = 2* precision * recall / (precision + recall)
  return f1

In [85]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
func = lambda trial: objective(trial, X_train, y_train, cv=kf)

study = optuna.create_study(direction="maximize")
study.optimize(func, n_trials=30)

[I 2024-02-16 15:14:51,537] A new study created in memory with name: no-name-9c0dfc56-52b1-4c3e-b675-00cded0813a6
[I 2024-02-16 15:17:23,589] Trial 0 finished with value: 0.8228944324255624 and parameters: {'n_estimators': 475, 'max_depth': 44, 'min_samples_split': 13, 'min_samples_leaf': 53}. Best is trial 0 with value: 0.8228944324255624.
[I 2024-02-16 15:18:33,395] Trial 1 finished with value: 0.8403788382442026 and parameters: {'n_estimators': 161, 'max_depth': 46, 'min_samples_split': 66, 'min_samples_leaf': 15}. Best is trial 1 with value: 0.8403788382442026.
[I 2024-02-16 15:22:16,963] Trial 2 finished with value: 0.8337860446712577 and parameters: {'n_estimators': 582, 'max_depth': 33, 'min_samples_split': 51, 'min_samples_leaf': 25}. Best is trial 1 with value: 0.8403788382442026.
[I 2024-02-16 15:26:26,737] Trial 3 finished with value: 0.8231336997058835 and parameters: {'n_estimators': 787, 'max_depth': 19, 'min_samples_split': 145, 'min_samples_leaf': 7}. Best is trial 1 wi

Проведем обучение модели с оптимизированными гиперпараметрами

In [ ]:
best_params = study.best_params
best_params

In [ ]:
rf_clf = RandomForestClassifier(**best_params)

In [ ]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=36, min_samples_leaf=2, min_samples_split=78,
                       n_estimators=510)

In [ ]:
X_array_train.shape

(25200, 230)

Оценим качество на отложенной выборке

In [ ]:
predictions = rf_clf.predict(X_array_test)

In [ ]:
f1 = f1_score(y_le_test, predictions, average='macro')
print(f'f1-score: {f1}')
print(classification_report(y_le_test, predictions))

f1-score: 0.857122555769295
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      6136
           1       0.99      0.73      0.84      1104
           2       0.96      0.67      0.79      1160

    accuracy                           0.91      8400
   macro avg       0.95      0.80      0.86      8400
weighted avg       0.92      0.91      0.91      8400



In [ ]:
print(confusion_matrix(y_le_test, predictions))

[[6095    7   34]
 [ 300  804    0]
 [ 384    0  776]]


## Интерпретация модели
Датасет имеет большое количество признаков. поэтому будеи рассматривать тоьлко наиболее значимую часть

In [ ]:
features = {}
for i in range(rf_clf.n_features_):
    features[rf_clf.feature_names_in_[i]] = rf_clf.feature_importances_[i]

In [ ]:
features

{'u1_0': 0.00041967266522166725,
 'u1_1': 0.0005361391752094259,
 'u1_2': 0.0003984252408352417,
 'u1_3': 0.0004805775154528018,
 'u1_4': 0.0004615941504428159,
 'u1_5': 0.00038768396359169666,
 'u1_6': 0.00044600732600419586,
 'u1_7': 0.000437736767710655,
 'u1_8': 0.0004285856517810288,
 'u1_9': 0.0004699863513966789,
 'u2_0': 0.00040832953748847953,
 'u2_1': 0.00034139089963497514,
 'u2_2': 0.0003689513600388665,
 'u2_3': 0.00042475707535082383,
 'u2_4': 0.0004078788858094017,
 'u2_5': 0.00046315222432194046,
 'u2_6': 0.0005403446011093487,
 'u2_7': 0.0005102543044194392,
 'u2_8': 0.0005004449697599839,
 'u2_9': 0.00041337975281424737,
 'fault_u2_0': 0.0,
 'fault_u2_1': 0.0,
 'fault_u2_2': 0.0,
 'fault_u2_3': 0.0,
 'fault_u2_4': 0.0,
 'fault_u2_5': 0.0,
 'fault_u2_6': 0.0,
 'fault_u2_7': 0.0,
 'fault_u2_8': 0.0,
 'fault_u2_9': 0.0,
 'dev_y1_0': 0.002184343039979027,
 'dev_y1_1': 0.0029385104626667886,
 'dev_y1_2': 0.0028710304996008634,
 'dev_y1_3': 0.006003500808737165,
 'dev_y1_4'

#### Mean Decrease Impurity

In [ ]:


forest_importances = pd.Series(rf_clf.feature_importances_, index=X_array_train.columns)

fig, ax = plt.subplots()
std = np.std([tree.feature_importances_ for tree in rf_clf.estimators_], axis=0)
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity", )
fig.tight_layout()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'